# Distribuzione predittiva a posteriori {#sec-post-pred-distr-discrete}

**Prerequisiti**

- Leggere il capitolo [Posterior Inference & Prediction](https://www.bayesrulesbook.com/chapter-8#ch8-post-pred) di [Bayes Rules!](https://www.bayesrulesbook.com).

**Concetti e competenze chiave**

- Imparare a combinare le probabilità a posteriori con le probabilità condizionali di un nuovo evento (come ottenere "croce" in un ulteriore lancio) per fare previsioni future. La distribuzione predittiva a posteriori ci permette di calcolare la probabilità di nuovi dati, tenendo conto delle evidenze osservate.
- Capire come la modellazione probabilistica aiuti a formalizzare l'incertezza e a fare previsioni basate su evidenze aggiornate, un concetto chiave nel ragionamento statistico e inferenziale.

**Preparazione del Notebook**

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats
import seaborn as sns
import arviz as az
import warnings

warnings.filterwarnings("ignore")

In [3]:
# set seed to make the results fully reproducible
seed: int = sum(map(ord, "poisson-model"))
rng: np.random.Generator = np.random.default_rng(seed=seed)
sns.set_theme(palette="colorblind")
az.style.use("arviz-darkgrid")

%config InlineBackend.figure_format = "retina"

## Introduzione

In questo capitolo esploreremo il concetto di *distribuzione predittiva a posteriori*, concentrandoci sul caso discreto. Per illustrare questo concetto in modo chiaro e intuitivo, utilizzeremo il problema del "bag of coins" come esempio esplicativo.

## Previsioni su Eventi Futuri

La *distribuzione predittiva a posteriori* è un concetto chiave nell'inferenza bayesiana. In parole semplici, rappresenta le probabilità degli eventi futuri basate su ciò che abbiamo già osservato. In altre parole, è come fare previsioni su quello che potrebbe succedere, utilizzando le informazioni raccolte fino a quel momento e le nostre convinzioni aggiornate (a posteriori).

## Definizione

Immagina di avere un insieme di dati che hai già osservato, chiamato $y = \{y_1, y_2, \ldots, y_n\}$. Supponiamo che questi dati siano stati generati da un modello con un parametro sconosciuto $\theta$, e che tu abbia una qualche idea iniziale (detta *a priori*) su quale possa essere il valore di $\theta$, rappresentata dalla distribuzione $p(\theta)$.

Dopo aver osservato i dati, possiamo aggiornare la nostra idea iniziale per ottenere una nuova distribuzione per $\theta$, chiamata *distribuzione a posteriori*, indicata come $p(\theta | y)$. Questa distribuzione riflette quello che abbiamo imparato sui parametri del modello dopo aver visto i dati.

La formula per la distribuzione a posteriori è:

$$
p(\theta | y) = \frac{p(y | \theta) p(\theta)}{p(y)},
$$

dove:

- $p(\theta | y)$ è la distribuzione a posteriori del parametro $\theta$;
- $p(y | \theta)$ è la probabilità di osservare i dati dati i parametri, chiamata *verosimiglianza*;
- $p(\theta)$ è la distribuzione a priori del parametro $\theta$;
- $p(y)$ è la probabilità di osservare quei dati, chiamata anche *evidenza*.

Per fare previsioni su un nuovo dato $\tilde{y}$, usiamo la *distribuzione predittiva a posteriori*. Questa si ottiene combinando tutte le possibili ipotesi sui parametri del modello, pesandole per quanto le riteniamo probabili in base ai dati osservati:

$$
p(\tilde{y} | y) = \int p(\tilde{y} | \theta) p(\theta | y) \, d\theta.
$$

Se il parametro $\theta$ può assumere solo alcuni valori discreti, invece di un integrale si usa una somma:

$$
p(\tilde{y} | y) = \sum_{\theta} p(\tilde{y} | \theta) p(\theta | y).
$$

Questa distribuzione combina la nostra convinzione aggiornata sui parametri $\theta$ con la probabilità di osservare un nuovo dato $\tilde{y}$, dato un certo valore di $\theta$. In questo modo, possiamo fare previsioni considerando tutte le incertezze che abbiamo sui parametri del modello.

## Il Sacchetto di Monete

In questo esempio discutiamo un problema che consente di presentare il concetto di distribuzione predittiva a posteriori in un modo intuivito. Immaginiamo di avere un sacchetto contenente tre tipi diversi di monete. Questi tre tipi di monete hanno caratteristiche uniche riguardo la probabilità di ottenere "testa" o "croce" quando vengono lanciate:

1. Moneta di Tipo 0: Questa moneta dà sempre "croce". La probabilità di ottenere "testa" è 0.
2. Moneta di Tipo 1: Questa è una moneta equa. La probabilità di ottenere "testa" è 0.5, e quindi anche la probabilità di ottenere "croce" è 0.5.
3. Moneta di Tipo 2: Questa moneta dà sempre "testa". La probabilità di ottenere "testa" è 1.

Supponiamo di scegliere una moneta a caso dal sacchetto e di lanciarla quattro volte. In tutte e quattro le volte otteniamo "croce". Ora vogliamo calcolare la probabilità di ottenere "croce" al quinto lancio.

### Passo 1: Probabilità Iniziali

Prima di lanciare la moneta, non sappiamo quale tipo di moneta abbiamo estratto dal sacchetto. Poiché ogni moneta ha la stessa probabilità di essere scelta, attribuiamo una probabilità iniziale (prior) uguale a ciascun tipo di moneta:

- Probabilità di avere la Moneta di Tipo 0: $\frac{1}{3}$.
- Probabilità di avere la Moneta di Tipo 1: $\frac{1}{3}$.
- Probabilità di avere la Moneta di Tipo 2: $\frac{1}{3}$.

### Passo 2: Osservare i Risultati e Aggiornare le Probabilità

Dopo aver osservato quattro lanci che hanno dato tutti "croce", possiamo aggiornare le nostre probabilità di avere un certo tipo di moneta utilizzando queste informazioni.

- Moneta di Tipo 0: Se avessimo questa moneta, ottenere "croce" in tutti i lanci è ciò che ci aspetteremmo, dato che dà sempre "croce".
- Moneta di Tipo 1: Con questa moneta, ottenere quattro "croce" di fila è possibile ma piuttosto improbabile, dato che la probabilità di "croce" in ciascun lancio è solo 0.5.
- Moneta di Tipo 2: Con questa moneta, ottenere "croce" è impossibile perché dà sempre "testa".

Basandoci su queste osservazioni, è molto più probabile che la moneta scelta sia di Tipo 0. La probabilità di avere la Moneta di Tipo 1 è bassa, mentre la probabilità di avere la Moneta di Tipo 2 è zero.

### Passo 3: Calcolare le Probabilità Posteriori

Per calcolare le nuove probabilità (posteriori) per ogni tipo di moneta dopo aver visto quattro "croce", utilizziamo la formula di Bayes. Supponiamo che $D$ rappresenti il dato "croce, croce, croce, croce". Dopo i calcoli (che qui non dettagliamo, ma che presenteremo in seguito), otteniamo le seguenti probabilità posteriori:

- Probabilità a posteriori di avere la Moneta di Tipo 0: circa 0.941.
- Probabilità a posteriori di avere la Moneta di Tipo 1: circa 0.059.
- Probabilità a posteriori di avere la Moneta di Tipo 2: 0.

### Passo 4: Predire il Risultato del Quinto Lancio

Per prevedere la probabilità di ottenere "croce" al quinto lancio, combiniamo le probabilità posteriori di ogni tipo di moneta con la probabilità di ottenere "croce" specifica a ciascun tipo:

- Moneta di Tipo 0: Probabilità di "croce" = 1.
- Moneta di Tipo 1: Probabilità di "croce" = 0.5.
- Moneta di Tipo 2: Probabilità di "croce" = 0.

Calcoliamo ora la probabilità totale di ottenere "croce" al quinto lancio:

1. Contributo della Moneta di Tipo 0:  $0.941 \times 1 = 0.941$.
2. Contributo della Moneta di Tipo 1:  $0.059 \times 0.5 = 0.0295$.
3. Contributo della Moneta di Tipo 2:  $0 \times 0 = 0$.

Sommando queste probabilità:

$$
0.941 + 0.0295 + 0 = 0.9705.
$$

Nel seguito considereremo come svolgere tutti i calcoli. 

::: {#exm-}

Iniziamo con una distribuzione a priori uniforme per ciascun tipo di moneta, poiché inizialmente non sappiamo quale sia stata scelta:

$$
P(X = 0) = P(X = 1) = P(X = 2) = \frac{1}{3}.
$$

Dopo aver osservato una sequenza di lanci, ad esempio quattro "croce" (0, 0, 0, 0), possiamo aggiornare le nostre credenze sulla base dei dati.

Calcoliamo la distribuzione a posteriori per ciascun tipo di moneta dopo aver osservato i risultati dei lanci.

Verosimiglianza per ciascun tipo di moneta:

  - Moneta tipo 0: Poiché dà sempre "croce", la verosimiglianza di osservare quattro "croce" è $P(D | X = 0) = 1$.
  - Moneta tipo 1: Poiché è equa, la verosimiglianza di osservare quattro "croce" è $P(D | X = 1) = (0.5)^4 = 0.0625$.
  - Moneta tipo 2: Poiché dà sempre "testa", la verosimiglianza di osservare quattro "croce" è $P(D | X = 2) = 0$.

Calcolo della distribuzione a posteriori per ciascun tipo di moneta:
  
$$
P(X = 0 | D) = \frac{P(D | X = 0)P(X = 0)}{P(D)},
$$

$$
P(X = 1 | D) = \frac{P(D | X = 1)P(X = 1)}{P(D)},
$$

$$
P(X = 2 | D) = \frac{P(D | X = 2)P(X = 2)}{P(D)},
$$

dove $P(D) = P(D | X = 0)P(X = 0) + P(D | X = 1)P(X = 1) + P(D | X = 2)P(X = 2)$.

Sostituendo i valori:

$$
P(D) = \frac{1}{3} \times 1 + \frac{1}{3} \times 0.0625 + \frac{1}{3} \times 0 = \frac{1 + 0.0625}{3} = \frac{1.0625}{3}.
$$

Quindi, le distribuzioni a posteriori diventano:

$$
P(X = 0 | D) = \frac{\frac{1}{3}}{\frac{1.0625}{3}} = \frac{1}{1.0625} \approx 0.941,
$$

$$
P(X = 1 | D) = \frac{\frac{1}{3} \times 0.0625}{\frac{1.0625}{3}} = \frac{0.0625}{1.0625} \approx 0.059,
$$

$$
P(X = 2 | D) = 0.
$$

Ora possiamo calcolare la distribuzione predittiva a posteriori per il prossimo lancio di moneta.

La probabilità che il prossimo lancio sia "croce" è data dalla somma delle probabilità ponderate da ciascun tipo di moneta, basate sulle distribuzioni a posteriori:

$$
P(\text{croce nel prossimo lancio} | D) = P(X = 0 | D) \times 1 + P(X = 1 | D) \times 0.5 + P(X = 2 | D) \times 0.
$$

Inserendo i valori calcolati:

$$
P(\text{croce nel prossimo lancio} | D) = 0.941 \times 1 + 0.059 \times 0.5 + 0 \times 0 = 0.9705.
$$

Quindi, la distribuzione predittiva a posteriori ci dice che, dato che abbiamo osservato quattro "croce", la probabilità che il prossimo lancio sia "croce" è circa $0.9705$.

::: 

### Interpretazione

In questo esempio, abbiamo utilizzato la *distribuzione predittiva a posteriori* per fare una stima sulla probabilità di ottenere "croce" al quinto lancio. La distribuzione predittiva a posteriori rappresenta una combinazione delle nostre credenze aggiornate (probabilità posteriori) riguardo a quale moneta abbiamo selezionato, basate sui dati osservati (i primi quattro lanci), e le probabilità di ottenere "croce" associate a ciascun tipo di moneta. In altre parole, ci dice qual è la probabilità di un nuovo dato (il quinto lancio che dia "croce") tenendo conto delle evidenze raccolte fino a quel punto.

Nel contesto dell'esempio, la distribuzione predittiva a posteriori è  data dalla combinazione delle probabilità posteriori di avere ciascun tipo di moneta con le probabilità di ottenere "croce" con quella moneta. Questa distribuzione ci permette di fare previsioni future (come il risultato del quinto lancio) basandoci sulle informazioni aggiornate dalle osservazioni precedenti. Il risultato finale di 0.9705 rappresenta la probabilità di ottenere "croce" al prossimo lancio, tenendo conto delle evidenze osservate.

La distribuzione predittiva a posteriori è fondamentale nell'inferenza bayesiana perché permette di fare previsioni che considerano tutte le incertezze riguardo ai parametri del modello. Non ci basiamo solo sul valore stimato dei parametri, ma teniamo conto anche della nostra incertezza su questi valori. 

## Riflessioni Conclusive

Utilizzando il modello "bag of coins", possiamo vedere come le osservazioni passate influenzano le previsioni future attraverso la distribuzione predittiva a posteriori. Questo esempio mostra come, anche con una conoscenza iniziale limitata, possiamo aggiornare le nostre credenze in modo sistematico e fare previsioni ragionevoli basate sui dati raccolti.


## Esercizi

::: {#exr-bag-of-coins-1}

Utilizzando un set di dati diverso, ovvero tre "testa" seguite da una "croce", calcola la distribuzione predittiva a posteriori per il prossimo lancio.

:::
   
::: {#exr-bag-of-coins-2}

Consideriamo un modello in cui ci sono tre tipi di monete, ognuna con una probabilità diversa di dare "testa" o "croce":

1. Moneta di Tipo 0: Questa moneta dà sempre "croce". La probabilità di ottenere "testa" è 0.
2. Moneta di Tipo 1: Questa moneta ha una probabilità di 0.7 di ottenere "testa" e 0.3 di ottenere "croce".
3. Moneta di Tipo 2: Questa moneta dà sempre "testa". La probabilità di ottenere "testa" è 1.

Nel sacchetto ci sono: 2 monete di tipo 0, 1 moneta di tipo 1 e 1 moneta di tipo 2. Supponiamo di osservare la sequenza *croce, testa, croce*. Vogliamo calcolare la probabilità di ottenere "croce" al quarto lancio.

:::

## Informazioni sull'Ambiente di Sviluppo

In [22]:
%load_ext watermark
%watermark -n -u -v -iv -w -m

Last updated: Sun Jun 02 2024

Python implementation: CPython
Python version       : 3.12.3
IPython version      : 8.24.0

Compiler    : Clang 16.0.6 
OS          : Darwin
Release     : 23.4.0
Machine     : arm64
Processor   : arm
CPU cores   : 8
Architecture: 64bit

numpy     : 1.26.4
pymc      : 5.15.0
scipy     : 1.13.0
arviz     : 0.18.0
matplotlib: 3.8.4
pandas    : 2.2.2

Watermark: 2.4.3

